### Makine Öğrenmesi ile Twitter Duygu-Durum Analizi

**İş Problemi**

Twitter kullanıcıları tarafından atılan tweetlerin taşıdığı duygu kapsamında pozitif, negatif ve nötr olarak tahmin edilmesi. 

**Veri Seti Hikayesi**

**'tweet_labeled.csv'** veri seti, 2022 yılında Twitter'da atılan tweetleri, tweetlerin atıldığı tarihleri ve tweetlerin içerdiği duygu kapsamında -1, 0 ve 1 olacak şekilde etiketleri içermektedir. **'tweets_2021.csv'** veri seti ise 2021 yılında atılan tweetleri kapsamaktadır.

**Değişkenler**

**tweet_id:** Tweet'in id bilgisi

**tweet:** Tweet içeriği

**date:** Tweetin atıldığı tarih ve saat bilgisi

**label:** Tweetin duygusuna göre el ile girilmiş etiket bilgisi (-1: negatif, 1: pozitif, 0: nötr)

In [1]:
# Gerekli Kütüphane ve Fonksiyonların Kurulması

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)

#### Görev 1: Feature Engineering

**Adım 1:** 'tweets_labeled.csv' adlı dosyayı okutunuz.

In [2]:
df = pd.read_csv("datasets/tweets_labeled.csv")

df.head()

,tweet_id,tweet,date,label
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-24 21:41:37+00:00,1
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-15 21:00:18+00:00,1
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-13 21:21:36+00:00,1
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 15:03:47+00:00,1
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 20:22:32+00:00,1


In [3]:
df.shape

(12960, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  12960 non-null  int64 
 1   tweet     12959 non-null  object
 2   date      12960 non-null  object
 3   label     12960 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 405.1+ KB


**Adım 2:** Eksik değer analizi yapınız, eksik değerler için gerekli işlemleri uygulayınız.

In [5]:
df.isnull().sum()

tweet_id    0
tweet       1
date        0
label       0
dtype: int64

In [6]:
df.dropna(inplace = True)

In [7]:
df.isnull().sum()

tweet_id    0
tweet       0
date        0
label       0
dtype: int64

**Adım 3:** 'date' değişkeninin saat dilimi GMT+03:00 olacak şekilde güncelleyiniz.

In [8]:
# date değişkeninin zaman diliminin İstanbul zaman dilimine çevrilmesi

df["date"] = pd.to_datetime(df["date"])
df["date"] = df["date"].dt.tz_convert("Europe/Istanbul")
df["date"] = df["date"].dt.tz_localize(None)

In [9]:
df.head()

,tweet_id,tweet,date,label
0,1606767075984375808,Berk Ali (kedim) seni çok özledim. Ölmek için ...,2022-12-25 00:41:37,1
1,1537178207677448193,"Yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,1
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,1
3,1495413883166760960,"Öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,1
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,1


**Adım 4:** 'Date' değişkenini baz alarak aşağıdaki 3 değişkeni oluşturunuz.

- mevsim
- gün
- 4 saatlik periyotlar (örneğin: 22:00-02:00, 02:00-06:00, 06:00-10:00, 10:00-14:00, 18:00-22:00)

In [10]:
# "month" değişkeninin oluşturulması ve düzenlenmesi

df["month"] = df["date"].dt.month_name()

df["tweet"] = df["tweet"].str.lower()

df["month"] = df['month'].replace({'December' : 'Aralık',
                                   'January'  : 'Ocak',
                                   'February' : 'Şubat',
                                   'March'    : 'Mart',
                                   'April'    : 'Nisan',
                                   'May'      : 'Mayıs',
                                   'June'     : 'Haziran',
                                   'July'     : 'Temmuz',
                                   'August'   : 'Ağustos',
                                   'September': 'Eylül',
                                   'October'  : 'Ekim',
                                   'November' : 'Kasım'})

In [11]:
df.head()

,tweet_id,tweet,date,label,month
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,1,Aralık
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,1,Haziran
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,1,Haziran
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,1,Şubat
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,1,Şubat


In [12]:
# "seasons" değişkeninin oluşturulması

seasons = {'Ocak'   : 'Kış',
           'Şubat'  : 'Kış',
           'Mart'   : 'İlkbahar',
           'Nisan'  : 'İlkbahar',
           'Mayıs'  : 'İlkbahar',
           'Haziran': 'Yaz',
           'Temmuz' : 'Yaz',
           'Ağustos': 'Yaz',
           'Eylül'  : 'Sonbahar',
           'Ekim'   : 'Sonbahar',
           'Kasım'  : 'Sonbahar',
           'Aralık' : 'Kış'}

In [13]:
df["seasons"] = df["month"].map(seasons)

df.head()

,tweet_id,tweet,date,label,month,seasons
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,1,Aralık,Kış
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,1,Haziran,Yaz
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,1,Haziran,Yaz
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,1,Şubat,Kış
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,1,Şubat,Kış


In [14]:
# gün değişkeninin oluşturulması

df["days"] = [date.strftime('%A') for date in df["date"]]

df["days"] = df["days"].replace({"Monday"   : "Pazartesi",
                                 "Tuesday"  : "Salı",
                                 "Wednesday": "Çarşamba",
                                 "Thursday" : "Perşembe",
                                 "Friday"   : "Cuma",
                                 "Saturday" : "Cumartesi",
                                 "Sunday"   : "Pazar"})

In [15]:
df.head()

,tweet_id,tweet,date,label,month,seasons,days
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,1,Aralık,Kış,Pazar
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,1,Haziran,Yaz,Perşembe
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,1,Haziran,Yaz,Salı
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,1,Şubat,Kış,Pazar
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,1,Şubat,Kış,Pazar


In [16]:
# 4 saatlik aralıklarla günün altıya bölünmesi

df["hour"] = df["date"].dt.hour

df["4_hour_interval"] = (df["hour"] // 2) * 2

interval = {0 : '0-2',
            2 : '2-4',
            4 : '4-6',
            6 : '6-8',
            8 : '8-10',
            10: '10-12',
            12: '12-14',
            14: '14-16',
            16: '16-18',
            18: '18-20',
            20: '20-22',
            22: '22-24'}

df["4_hour_interval"] = df["4_hour_interval"].map(interval)

df["time_interval"] = df["4_hour_interval"].replace({"0-2"  : "22-02",
                                                     "22-24": "22-02",
                                                     "2-4"  : "02-06",
                                                     "4-6"  : "02-06",
                                                     "6-8"  : "06-10",
                                                     "8-10" : "06-10",
                                                     "10-12": "10-14",
                                                     "12-14": "10-14",
                                                     "14-16": "14-18",
                                                     "16-18": "14-18",
                                                     "18-20": "18-22",
                                                     "20-22": "18-22"})

In [17]:
df.head()

,tweet_id,tweet,date,label,month,seasons,days,hour,4_hour_interval,time_interval
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,1,Aralık,Kış,Pazar,0,0-2,22-02
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,1,Haziran,Yaz,Perşembe,0,0-2,22-02
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,1,Haziran,Yaz,Salı,0,0-2,22-02
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,1,Şubat,Kış,Pazar,18,18-20,18-22
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,1,Şubat,Kış,Pazar,23,22-24,22-02


In [18]:
df.drop(["4_hour_interval", "hour"], axis = 1, inplace = True)

df.head()

,tweet_id,tweet,date,label,month,seasons,days,time_interval
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,1,Aralık,Kış,Pazar,22-02
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,1,Haziran,Yaz,Perşembe,22-02
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,1,Haziran,Yaz,Salı,22-02
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,1,Şubat,Kış,Pazar,18-22
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,1,Şubat,Kış,Pazar,22-02


#### Görev 2: Veriyi Hazırlama ve Logistic Regression

**Adım 1:** 'tweet' değişkeninin harflerinin küçük harfe çeviriniz.

In [19]:
df["tweet"] = df["tweet"].str.lower()

df["tweet"].head()

0    berk ali (kedim) seni çok özledim. ölmek için ...
1    yani, öylesine ciddiye alacaksın ki yaşamayı, ...
2              saçlarının gölgesinde\nölmek ne güzeldi
3    öyle güzel baktın ki, gözlerime\nsevmek değil ...
4    sevmek biri için ölmek değil her şeye rağmen y...
Name: tweet, dtype: object

**Adım 2:** label encoding işlemi uygulayarak 'label' değişkenini sayısal olarak ifade ediniz.

In [20]:
df["label"].replace(1, value = "pozitif", inplace = True)
df["label"].replace(-1, value = "negatif", inplace = True)
df["label"].replace(0, value = "nötr", inplace = True)

df.head()

,tweet_id,tweet,date,label,month,seasons,days,time_interval
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,pozitif,Aralık,Kış,Pazar,22-02
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,pozitif,Haziran,Yaz,Perşembe,22-02
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,pozitif,Haziran,Yaz,Salı,22-02
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,pozitif,Şubat,Kış,Pazar,18-22
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,pozitif,Şubat,Kış,Pazar,22-02


In [21]:
df["label"] = LabelEncoder().fit_transform(df["label"])

df.head()

,tweet_id,tweet,date,label,month,seasons,days,time_interval
0,1606767075984375808,berk ali (kedim) seni çok özledim. ölmek için ...,2022-12-25 00:41:37,2,Aralık,Kış,Pazar,22-02
1,1537178207677448193,"yani, öylesine ciddiye alacaksın ki yaşamayı, ...",2022-06-16 00:00:18,2,Haziran,Yaz,Perşembe,22-02
2,1536458790802972673,saçlarının gölgesinde\nölmek ne güzeldi,2022-06-14 00:21:36,2,Haziran,Yaz,Salı,22-02
3,1495413883166760960,"öyle güzel baktın ki, gözlerime\nsevmek değil ...",2022-02-20 18:03:47,2,Şubat,Kış,Pazar,18-22
4,1490420667614904334,sevmek biri için ölmek değil her şeye rağmen y...,2022-02-06 23:22:32,2,Şubat,Kış,Pazar,22-02


**Adım 3:** 'tweet' değişkenini kullanarak TF-IDF matrisini oluşturunuz.

In [22]:
X = TfidfVectorizer().fit_transform(df["tweet"])
y = df["label"]

**Adım 4:** TF-IDF matrisini ve 'label' hedef değişkenini kullanarak logistic regression modelini kurunuz ve accuracy'yi değerlendiriniz.

In [23]:
log_model = LogisticRegression(max_iter = 10000).fit(X, y)

print(cross_val_score(log_model,
                X,
                y,
                scoring = "accuracy",
                cv = 10).mean())

0.6705764097430765


**Adım 5:** 'tweets_2021.csv' adlı dosyayı okutunuz, değişken tiplerini düzenleyiniz.

In [24]:
df_new = pd.read_csv("datasets/tweets_21.csv")

df_new.head()

,tweet_id,tweet,date
0,1344798002490314752,Kardesim (12) sevdigi cocuga 00:00 da ilan-i a...,2021-01-01 03:10:03
1,1344799527673470977,Bu yılbaşı da saat tam 00:00'da swni seviyorum...,2021-01-01 03:16:07
2,1344799907719348226,tam 00.00da naptınız ben her yıl tam bu vakit ...,2021-01-01 03:17:37
3,1344800782802165762,00:00'da havai fişek gösterisi yapıldı tam dib...,2021-01-01 03:21:06
4,1344805589990453249,31 aralık saat 00.00 da yeni yılımızı kutlayan...,2021-01-01 03:40:12


In [25]:
df_new.shape

(13272, 3)

In [26]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13272 entries, 0 to 13271
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  13272 non-null  int64 
 1   tweet     13272 non-null  object
 2   date      13272 non-null  object
dtypes: int64(1), object(2)
memory usage: 311.2+ KB


In [27]:
df_new.isnull().sum()

tweet_id    0
tweet       0
date        0
dtype: int64

**Adım 6:** tweet'leri küçük harfe çeviriniz.

In [28]:
df_new["tweet"] = df_new["tweet"].str.lower()

df_new["tweet"].head()

0    kardesim (12) sevdigi cocuga 00:00 da ilan-i a...
1    bu yılbaşı da saat tam 00:00'da swni seviyorum...
2    tam 00.00da naptınız ben her yıl tam bu vakit ...
3    00:00'da havai fişek gösterisi yapıldı tam dib...
4    31 aralık saat 00.00 da yeni yılımızı kutlayan...
Name: tweet, dtype: object

#### Görev 3: Tweetlerde Duygu Tahmini

**Adım 1:** 'tf_idf_vectorizer' ve logistic regression modelini kullanarak 2021 yılı için tahmin işlemini gerçekleştiriniz.

In [29]:
tweet_tf_idf = TfidfVectorizer()

tweet_tf_idf.fit(df["tweet"])

TfidfVectorizer()

In [30]:
X_new = tweet_tf_idf.transform(df_new["tweet"])

In [31]:
predictions = log_model.predict(X_new)

**Adım 2:** 'label' adında bir değişken oluşturunuz ve bu tahminleri bu değişkenin altına kaydediniz.

In [32]:
df_new["label"] = predictions

In [33]:
df_new.head()

,tweet_id,tweet,date,label
0,1344798002490314752,kardesim (12) sevdigi cocuga 00:00 da ilan-i a...,2021-01-01 03:10:03,1
1,1344799527673470977,bu yılbaşı da saat tam 00:00'da swni seviyorum...,2021-01-01 03:16:07,1
2,1344799907719348226,tam 00.00da naptınız ben her yıl tam bu vakit ...,2021-01-01 03:17:37,1
3,1344800782802165762,00:00'da havai fişek gösterisi yapıldı tam dib...,2021-01-01 03:21:06,1
4,1344805589990453249,31 aralık saat 00.00 da yeni yılımızı kutlayan...,2021-01-01 03:40:12,1


In [34]:
df_new["label"].value_counts()

label
1    11468
0      964
2      840
Name: count, dtype: int64